### 1. 多项目构建目录树
1. build.sbt  
 定义工程
2. project/plugins.sbt  
 定义插件, 用于打包
3. 子工程目录树  
 子工程类文件保持在src/main/scala下, 否则打包时识别不出类文件
<img src="../img/multisbt.png" width="35%" height="35%">

#### 2. project/plugins.sbt
```scala
addSbtPlugin("com.eed3si9n" % "sbt-assembly" % "0.14.5")
```

#### 3. build.sbt
```scala
// 完整内容
val spark_version = "2.1.1"
val sparkCore = "org.apache.spark" % "spark-core_2.11" % spark_version
val sparkStreaming = "org.apache.spark" % "spark-streaming_2.11" % spark_version
val sparkSql = "org.apache.spark" % "spark-sql_2.11" % spark_version
val sparkHive = "org.apache.spark" % "spark-hive_2.11" % spark_version
val sparkMLib = "org.apache.spark" % "spark-mllib_2.11" % spark_version
val config = "com.typesafe" % "config" % "1.3.2"

// 依赖,打包设置
lazy val sparkDependencies = Seq(sparkCore,sparkStreaming,sparkSql,sparkHive,sparkMLib).map(_ % "provided")
lazy val commonDependencies = Seq(config)
lazy val assemblySettings = Seq(
  assemblyOption in assembly := (assemblyOption in assembly).value.copy(includeScala = false),
  assemblyMergeStrategy in assembly := {
    case PathList("play", "core", xs @ _*) => MergeStrategy.first
    case x =>
      val oldStrategy = (assemblyMergeStrategy in assembly).value
      oldStrategy(x)
  }
)
lazy val commonSettings = Seq(version := "0.1",
  scalaVersion := "2.11.8"
)

// moudle定义,加入依赖和打包设置
lazy val sub1 = (project in file("sub1"))
  .settings(commonSettings:_*)
  .settings(
    name := "sub1",
    assemblySettings,
    libraryDependencies ++= sparkDependencies,
    libraryDependencies  ++= commonDependencies
  )//.dependsOn(sub2)
lazy val root = (project in file("."))
  .settings(commonSettings:_*)
  .settings(
    name := "root",
    assemblySettings
  ).aggregate(sub1)
```